In [ ]:
#libraries for OCR

!pip install flask-ngrok
!pip install tensorflow-io

!pip install textblob 


#OCR tesserac
!sudo apt install tesseract-ocr
!pip install pytesseract

# add new language to pytesseract
try:
  !wget https://github.com/tesseract-ocr/tessdata/blob/main/fra.traineddata
  !wget https://github.com/tesseract-ocr/tessdata/blob/main/deu.traineddata
  !wget https://github.com/tesseract-ocr/tessdata/blob/main/ita.traineddata

  !sudo mv -v *.traineddata /usr/share/tesseract-ocr/4.00/tessdata/

  print("new language installed")
except:
  pass


In [ ]:
## tokenizer_conf_v3.txt : containing the dictionary for text generation
## CaptionModel.py: containing model and function for image captioning

##if necessary you can copy the files to the workspace from your drive, or simply import them with the drag drop

#from google.colab import drive
#drive.mount('/content/drive')

#!cp  /content/drive/MyDrive/DeepVision/tokenizer_conf_v3.txt /content/

#!cp /content/drive/MyDrive/DeepVision/CaptionModel.py  /content/


Mounted at /content/drive


In [ ]:
import tensorflow as tf
import numpy as np
import random
from PIL import ImageEnhance, ImageFilter, Image


import matplotlib.pyplot as plt
from matplotlib.figure import Figure




from flask import Flask, jsonify, request,make_response
from flask_ngrok import run_with_ngrok
import json


import importlib

from textblob import TextBlob
import re

import pytesseract
!which tesseract
pytesseract.pytesseract.tesseract_cmd = (r'/usr/bin/tesseract')

#/usr/bin/tesseract
print(pytesseract.get_languages(config=''))

/usr/bin/tesseract
['deu', 'osd', 'ita', 'fra', 'eng']


In [ ]:
# Check available GPU devices.
pyVersion = !python --version
print('Using Tensorflow version:', tf.__version__, '\nwith GPU support:', tf.config.list_physical_devices('GPU'), "\npython version : ", pyVersion )

Using Tensorflow version: 2.5.0 
with GPU support: [] 
python version :  ['Python 3.7.10']


In [ ]:
import CaptionModel as module
importlib.reload(module)

In [ ]:
def postProcessOcr(txt):
  txt = txt.encode('utf-8')
  txt = str(txt.decode())
  txt = re.sub(r"\n", " ", txt)
  txt = re.sub(r"([?.!,¿])", r"", txt)
  txt = re.sub(r"\x0c", " ", txt)
  return txt

In [ ]:
app = Flask(__name__)

run_with_ngrok(app)


In [ ]:
#langueCode = {"English":"en", "Chinese":"cn", "French":"fr", "German":"de", "Hindi":"in", "Russian":"ru", "Spanish":"es", "Arabic":"qa"}

In [ ]:


@app.route("/test", methods=['GET', 'POST'])
def Imgtest():
  
    return "hello word from Google Colab"
  


#@app.route('/imageclassifier/predict/', methods=['GET', 'POST'])
@app.route("/hello", methods=['GET', 'POST'])
def image_classifier():

    imgfile = request.files['img_file']
    print(imgfile)
    img2 = Image.open(imgfile)

   

    try:
      param_lang = request.form["param_lang"] 
      param_process = request.form["param_process"] #"phoneCam"; glass1
      mode = request.form["mode"]
      SensorAngle = int(request.form["SensorAngle"])
      

    except:
      param_lang = request.args.get("param_lang")
      param_process = request.form["param_process"]



    img2 = img2.rotate(-SensorAngle, resample=Image.BICUBIC, expand=True)
    
    
    img2 = np.array(img2)
    print(img2.shape)
    plt.imshow(img2)
    plt.show()
      

    
    
   ####run model
    if mode == "caption":

     text_en = module.executeModel(img2, param_process)
    
    elif  mode == "ocr":
     text_en = pytesseract.image_to_string(img2)#, lang='eng')
     text_en = postProcessOcr(text_en)

    


    #### translation 
    if param_lang != "English":  #support only English and French for the moment
      text_en = str(TextBlob(text_en).translate(from_lang='en', to='fr'))

    return text_en

    #return jsonify(result_class = "class_name")

In [ ]:
if __name__ == "__main__":
  app.run()
  